In [1]:
import warnings
import itertools
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import datetime

plt.style.use('fivethirtyeight')

In [2]:
cote_divoire = pd.read_csv("cote_divoire_growth.csv", index_col="year", parse_dates=True)
cote_divoire.head()

cote_divoire.rename({"Cote d'Ivoire":"pop_growth_rate"},axis=1, inplace=True)

cote_divoire.loc["2018-01-01","pop_growth_rate"] = cote_divoire.loc["2017-01-01", "pop_growth_rate"]

cote_divoire.head()

,pop_growth_rate
year,
1960-01-01,3.518860
1961-01-01,3.728914
1962-01-01,3.898586
1963-01-01,3.977808
1964-01-01,3.949645


In [3]:
# Define the p, d and q parameters to take any value between 0 and 2
p = d = q = range(0, 6)

# Generate all different combinations of p, q and q triplets
pdq = list(itertools.product(p, d, q))

# Generate all different combinations of seasonal p, q and q triplets
seasonal_pdq = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))]

print('Examples of parameter combinations for Seasonal ARIMA...')
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[1]))
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[2]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[3]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[4]))

Examples of parameter combinations for Seasonal ARIMA...
SARIMAX: (0, 0, 1) x (0, 0, 1, 12)
SARIMAX: (0, 0, 1) x (0, 0, 2, 12)
SARIMAX: (0, 0, 2) x (0, 0, 3, 12)
SARIMAX: (0, 0, 2) x (0, 0, 4, 12)


In [ ]:
warnings.filterwarnings("ignore") # specify to ignore warning messages

L =[]
for param in pdq:
    for param_seasonal in seasonal_pdq:
        #print(param, param_seasonal)
        try:
            mod = sm.tsa.statespace.SARIMAX(cote_divoire,
                                            order=param,
                                            seasonal_order=param_seasonal,
                                            enforce_stationarity=False,
                                            enforce_invertibility=False)

            results = mod.fit()
            #print(results.summary().tables[1])
            
            y_truth = cote_divoire['2000-01-01':]
            y_truth = np.array(y_truth["pop_growth_rate"].tolist() )
            #print(y_truth)
            # Dynamic False
            pred = results.get_prediction(start=pd.to_datetime("2000-01-01"), dynamic=False)
            #pred_ci = pred.conf_int()
            
            y_forecasted = pred.predicted_mean
            y_forecasted = np.array(y_forecasted.tolist() )
            
            # Compute the mean square error
            mse1 = np.mean( ( y_forecasted - y_truth ) ** 2)
            
            # Dynamic True
            pred_dynamic = results.get_prediction(start=pd.to_datetime('2000-01-01'), dynamic=True, full_results=True)   
            #pred_dynamic_ci = pred_dynamic.conf_int()
            
            y_forecasted = pred_dynamic.predicted_mean
            y_forecasted = np.array(y_forecasted.tolist() )

            # Compute the mean square error
            mse = np.mean( ( y_forecasted - y_truth ) ** 2)
            
            #print(mse,mse1)
            if round(mse, 5) <= 0.005 and round(mse1, 5) <= 0.002:  
                L.append((results.aic,'ARIMA{}x{}12'.format(param, param_seasonal),mse,mse1 ))
                
                print('ARIMA{}x{}12 - AIC:{}'.format(param, param_seasonal, results.aic),mse,mse1)
        except:
            continue
                    
                
file_params = open("param_estimated.dat", 'w')           
file_params.write("\n".join([str(i)+";"+str(j)+";"+str(mse)+";"+str(mse1) for i,j,mse,mse1 in L]))
file_params.close()             

